In [ ]:
#importing the dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM , Embedding
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re

In [ ]:
#loading the dataset
dataset = pd.read_csv("/content/drive/MyDrive/datasets/Tweets.csv")

In [ ]:
#checking top 5 documents
dataset.head()


,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,name,retweet_count,text,tweet_created,latitude,longitude
0,5.703061e+17,neutral,1.0000,NaN,NaN,Virgin America,cairdin,0,@VirginAmerica What @dhepburn said.,2015-02-24 11:35:52 -0800,35.888455,-119.273781
1,5.703011e+17,positive,0.3486,NaN,0.0000,Virgin America,jnardino,0,@VirginAmerica plus you've added commercials t...,2015-02-24 11:15:59 -0800,37.770971,-119.941025
2,5.703011e+17,neutral,0.6837,NaN,NaN,Virgin America,yvonnalynn,0,@VirginAmerica I didn't today... Must mean I n...,2015-02-24 11:15:48 -0800,35.684863,-119.709299
3,5.703010e+17,negative,1.0000,Bad Flight,0.7033,Virgin America,jnardino,0,@VirginAmerica it's really aggressive to blast...,2015-02-24 11:15:36 -0800,37.061159,-119.279135
4,5.703008e+17,negative,1.0000,Can't Tell,1.0000,Virgin America,jnardino,0,@VirginAmerica and it's a really big bad thing...,2015-02-24 11:14:45 -0800,36.790587,-120.867752


In [ ]:
#to check which airlines reviews we have recived
dataset['airline'].unique()

array(['Virgin America', 'United', 'Southwest', 'Delta', 'US Airways',
       'American'], dtype=object)

In [ ]:
#what are the sentiment or labels to predict
dataset['airline_sentiment'].unique()

array(['neutral', 'positive', 'negative'], dtype=object)

In [ ]:
#checking for the null values
dataset.isnull().sum()

tweet_id                           0
airline_sentiment                  0
airline_sentiment_confidence       0
negativereason                  5462
negativereason_confidence       4118
airline                            0
name                               0
retweet_count                      0
text                               0
tweet_created                      0
latitude                           0
longitude                          0
dtype: int64

In [ ]:
#creating a new dataframe with only 2 columns the review and the respected sentiment  
data = dataset[["text","airline_sentiment"]]

In [ ]:
#new dataframe
data.head()

,text,airline_sentiment
0,@VirginAmerica What @dhepburn said.,neutral
1,@VirginAmerica plus you've added commercials t...,positive
2,@VirginAmerica I didn't today... Must mean I n...,neutral
3,@VirginAmerica it's really aggressive to blast...,negative
4,@VirginAmerica and it's a really big bad thing...,negative


In [ ]:
#removing all the rows where ever the sentiment is neutral we want to work with positive and negative reviews
dataset = data[data.airline_sentiment != 'neutral']

In [ ]:
dataset.head()

,text,airline_sentiment
1,@VirginAmerica plus you've added commercials t...,positive
3,@VirginAmerica it's really aggressive to blast...,negative
4,@VirginAmerica and it's a really big bad thing...,negative
5,@VirginAmerica seriously would pay $30 a fligh...,negative
6,"@VirginAmerica yes, nearly every time I fly VX...",positive


In [ ]:
dataset.airline_sentiment.unique()

array(['positive', 'negative'], dtype=object)

In [ ]:
dataset['text'][3]

'@VirginAmerica it\'s really aggressive to blast obnoxious "entertainment" in your guests\' faces &amp; they have little recourse'

In [ ]:
import nltk
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
#checking what all words are considered as stopwords
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
dataset.info

<bound method DataFrame.info of                                                     text airline_sentiment
1      @VirginAmerica plus you've added commercials t...          positive
3      @VirginAmerica it's really aggressive to blast...          negative
4      @VirginAmerica and it's a really big bad thing...          negative
5      @VirginAmerica seriously would pay $30 a fligh...          negative
6      @VirginAmerica yes, nearly every time I fly VX...          positive
...                                                  ...               ...
14633  @AmericanAir my flight was Cancelled Flightled...          negative
14634         @AmericanAir right on cue with the delays👌          negative
14635  @AmericanAir thank you we got on a different f...          positive
14636  @AmericanAir leaving over 20 minutes Late Flig...          negative
14638  @AmericanAir you have my money, you change my ...          negative

[11541 rows x 2 columns]>

In [ ]:
#converting all the text to lower case
dataset['text'] = dataset['text'].apply(lambda x:x.lower())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
print(dataset['text'].iloc[3])

@virginamerica seriously would pay $30 a flight for seats that didn't have this playing.
it's really the only bad thing about flying va


In [ ]:
#removing all the punctuations and symbols
dataset['text'] = dataset['text'].apply(lambda x: re.sub('[^a-zA-Z0-9\s]','',x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
print(dataset['text'].iloc[3])

virginamerica seriously would pay 30 a flight for seats that didnt have this playing
its really the only bad thing about flying va


In [ ]:
dataset.shape

(11541, 2)

In [ ]:
#loading the stopwords
stop_words = set(stopwords.words("english"))

In [ ]:
#removing the stop words from the corpus
dataset['text'] = dataset['text'].apply(lambda x:' '.join([word for word in x.split() if word not in (stop_words)]))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
print(dataset['text'].iloc[3])

virginamerica seriously would pay 30 flight seats didnt playing really bad thing flying va


In [ ]:
#tokenizing our document 
max_features = 2000
tokenizer = Tokenizer(num_words=max_features,split = ' ')
tokenizer.fit_on_texts(dataset['text'].values)

In [ ]:
print(dataset['text'].iloc[3])

virginamerica seriously would pay 30 flight seats didnt playing really bad thing flying va


In [ ]:
#converting our tokens or text to sequences
seq = tokenizer.texts_to_sequences(dataset['text'].values)

In [ ]:
len(seq[3])

14

In [ ]:
print(seq[3])

[43, 283, 34, 185, 186, 2, 121, 104, 1980, 56, 115, 376, 78, 1807]


In [ ]:
len(seq[12])

15

In [ ]:
print(seq[12])

[43, 769, 88, 1190, 710, 2, 242, 683, 47, 434, 146, 46, 383, 927, 1265]


In [ ]:
#Doing the padding operation so that all the documents are of same length 
seq = pad_sequences(seq)

In [ ]:
print(seq[12])

[   0    0    0    0    0    0    0   43  769   88 1190  710    2  242
  683   47  434  146   46  383  927 1265]


In [ ]:
print(seq[20])

[   0    0    0    0    0    0    0    0    0    0    0    0   43  296
 1073 1074 1652  321  315  165 1137  101]


In [ ]:
seq.shape

(11541, 22)

In [ ]:
#converting the label to numeric form
y = pd.get_dummies(dataset['airline_sentiment'],drop_first=True).values

In [ ]:
y[3]

array([0], dtype=uint8)

In [ ]:
#splitting the data into train and test
x_train,x_test,y_train,y_test = train_test_split(seq, y, test_size = 0.2,random_state = 0)

In [ ]:
x_train.shape

(9232, 22)

In [ ]:
x_train[3]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          1,   21,  698, 1485,   88,  239,    2, 1827,   46,   10, 1938],
      dtype=int32)

In [ ]:
y_train[3]

array([1], dtype=uint8)

In [ ]:
#giving the embedding dimension, the size of the dimension which we want to convert our word into
embed_dimension = 128

In [ ]:
#Building the model 
model = Sequential()
model.add(Embedding(max_features,embed_dimension,input_length=seq.shape[1])) #Embedding layer - input_Dimension, embed_Dim we need, and the input length of the document
model.add(LSTM(128, dropout=0.2, recurrent_dropout= 0.2)) #buildinf a lstm layer 
model.add(Dense(1,activation = 'sigmoid')) #output layer

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 22, 128)           256000    
_________________________________________________________________
lstm (LSTM)                  (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 387,713
Trainable params: 387,713
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#compiling the model
model.compile(loss = 'binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
#training our model and checking the validation accuracy
model.fit(x_train,y_train,batch_size=100,validation_data=(x_test,y_test),epochs=10)

Epoch 1/10
93/93 [==============================] - 13s 91ms/step - loss: 0.5107 - accuracy: 0.7974 - val_loss: 0.2267 - val_accuracy: 0.9199
Epoch 2/10
93/93 [==============================] - 8s 87ms/step - loss: 0.1860 - accuracy: 0.9319 - val_loss: 0.1959 - val_accuracy: 0.9311
Epoch 3/10
93/93 [==============================] - 8s 85ms/step - loss: 0.1451 - accuracy: 0.9456 - val_loss: 0.1999 - val_accuracy: 0.9242
Epoch 4/10
93/93 [==============================] - 8s 84ms/step - loss: 0.1113 - accuracy: 0.9595 - val_loss: 0.2113 - val_accuracy: 0.9173
Epoch 5/10
93/93 [==============================] - 8s 86ms/step - loss: 0.1028 - accuracy: 0.9615 - val_loss: 0.2325 - val_accuracy: 0.9194
Epoch 6/10
93/93 [==============================] - 8s 85ms/step - loss: 0.0866 - accuracy: 0.9685 - val_loss: 0.2448 - val_accuracy: 0.9212
Epoch 7/10
93/93 [==============================] - 8s 86ms/step - loss: 0.0788 - accuracy: 0.9728 - val_loss: 0.2997 - val_accuracy: 0.9164
Epoch 8/10
9

In [ ]:
#predicting a random tweet 
tweets = ["virginamerica has the most incredible customer service i've ever experienced! so refreshing!"]

In [ ]:
from nltk import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
#tokenized_twt = word_tokenize(tweet)

In [ ]:
#tokenizing the tweet
tokenizer.fit_on_texts(tweets)

In [ ]:
#removing stop words from the tweet
filtered_twts  =[]
for word in tweets:
  if word not in stop_words:
    filtered_twts.append(word)

In [ ]:
filtered_twts

["virginamerica has the most incredible customer service i've ever experienced! so refreshing!"]

In [ ]:
#converting text to sequence
seq1 = tokenizer.texts_to_sequences(filtered_twts)

In [ ]:
seq1

[[43, 1286, 11, 10, 77, 898]]

In [ ]:
#padding our document according to shape of our data
seq1 = pad_sequences(seq1, maxlen=22)

In [ ]:
seq1

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,   43, 1286,   11,   10,   77,  898]],
      dtype=int32)

In [ ]:
#predicting the sentiment of the tweet
y_pred = model.predict_classes(seq1)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [ ]:
y_pred

array([[1]], dtype=int32)